## Check if we have dbatools docker images

In [1]:
docker images "dbatools/*"

## Verify if we have containers setup

In [2]:
docker ps -a

### If containers are not up and running, start them

In [3]:
# start container dockersql1
docker start dockersql1

# start container dockersql2 
docker start dockersql2

### If not available we need to start them with parameters

In [3]:
# stop and remove containers
docker kill dockersql1, dockersql2
docker rm dockersql1, dockersql2

# setup two containers and expose ports
docker run `
    -p 1433:1433 `
    -p 5022:5022 `
    --network localnet `
    -v sharedFolder:/tmp `
    --hostname dockersql1 `
    --name dockersql1 `
    -d dbatools/sqlinstance
    

docker run `
    -p 14333:1433 `
    -p 5023:5022 `
    --network localnet `
    -v sharedFolder:/tmp `
    --hostname dockersql2 `
    --name dockersql2 `
    -d dbatools/sqlinstance2

## Set variables values

In [4]:
$dbatools1 = "localhost,1433"
$dbatools2 = "localhost,14333"
$database = "Pubs"
$AG = "test-ag"
$secureString = ConvertTo-SecureString "dbatools.IO" -AsPlainText -Force
$cred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "sqladmin", $secureString

## Remove 'test-ag' from AG

In [5]:
# Remove Database from AG
Remove-DbaAgDatabase -SqlInstance $dbatools1 -SqlCredential $cred -AvailabilityGroup $AG -Database $database -Confirm:$false

# Remove Availability group
Remove-DbaAvailabilityGroup -SqlInstance $dbatools1, $dbatools2 -SqlCredential $cred -AvailabilityGroup $AG -Confirm:$false

# Remove Database from secondary instance
Remove-DbaDatabase -SqlInstance $dbatools2 -SqlCredential $cred -Database $database -Confirm:$false

# Grant permissions

In [6]:
$filePermissions = "D:\Presentations\SQLSaturday#926 - Lisboa 2019\Reset\02_GrantPermissions.sql"

$scriptWithoutGO = (Get-Content $filePermissions -Raw) -replace '\bGO\b', ' '

Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Query $scriptWithoutGO -Verbose

### Check Permissions

In [7]:
Get-DbaUserPermission -SqlInstance $dbatools1 -SqlCredential $cred

### Remove files from directories

In [9]:
Remove-Item "D:\Presentations\SQLSaturday#926 - Lisboa 2019\Replicate Permissions\*.sql" -Confirm:$false

# Remove temp folder content

In [1]:
Remove-Item "D:\Presentations\SQLSaturday#926 - Lisboa 2019\Export-DbaDatabase\temp\*" -Recurse -Confirm:$false -Force

Remove-Item "D:\Presentations\SQLSaturday#926 - Lisboa 2019\Export-DbaDatabase\*" -Exclude "temp" -Recurse -Confirm:$false -Force

<hr>

## Reset for copy data demo

Remove `EmptyNorthwind` database from `localhost,14333` instance

In [9]:
Remove-DbaDatabase -SqlInstance $dbatools2 -SqlCredential $cred -Database EmptyNorthwind -Confirm:$false

Drop table `[dbo].[CopyOf_Order Details]`

In [10]:
Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Database Northwind -Query "DROP TABLE [dbo].[CopyOf_Order Details]"

# Reset for refresh database demo
### Create `dbatools` database from `pubs` on `dockersql1` and also copy to `dockersql2`
### They will have different users

In [11]:
# Remove database if exists
Remove-DbaDatabase -SqlInstance $dbatools1, $dbatools2 -SqlCredential $cred -Database "dbatools" -Confirm:$false

#Same instance
$copyDatabaseSplat = @{
    Source = $dbatools1
    SourceSqlCredential = $cred
    Destination = $dbatools1
    DestinationSqlCredential = $cred
    Database = "pubs"
    BackupRestore = $true
    SharedPath = "/tmp" 
    NewName = "dbatools"
}
Copy-DbaDatabase @copyDatabaseSplat

# Create login/user with "_qa"
$loginUser_qa = "dbatools_qa"
$refreshDatabase = "dbatools"
New-DbaLogin -SqlInstance $dbatools1 -SqlCredential $cred -Login $loginUser_qa -SecurePassword $secureString
New-DbaDbUser -SqlInstance $dbatools1 -SqlCredential $cred -Username $loginUser_qa -Login $loginUser_qa -Database $refreshDatabase

# Grant some permissions
$grants = "GRANT CREATE VIEW TO [dbatools_qa]  AS [dbo];
GRANT DELETE ON OBJECT::[dbo].[authors] TO [dbatools_qa]  AS [dbo];
GRANT INSERT ON OBJECT::[dbo].[authors] TO [dbatools_qa]  AS [dbo];
"
# Run the exported script
Invoke-DbaQuery -SqlInstance $dbatools1 -SqlCredential $cred -Database $refreshDatabase -Query $grants -Verbose

$copyDatabaseSplat = @{
    Source = $dbatools1
    SourceSqlCredential = $cred
    Destination = $dbatools2
    DestinationSqlCredential = $cred
    Database = "dbatools"
    BackupRestore = $true
    SharedPath = "/tmp"
}
Copy-DbaDatabase @copyDatabaseSplat


# Create login/user with "_dev"
$loginUser_dev = "dbatools_dev"
$refreshDatabase = "dbatools"
New-DbaLogin -SqlInstance $dbatools2 -SqlCredential $cred -Login $loginUser_dev -SecurePassword $secureString
New-DbaDbUser -SqlInstance $dbatools2 -SqlCredential $cred -Username $loginUser_dev -Login $loginUser_dev -Database $refreshDatabase

# Grant some permissions
$grants = "GRANT CREATE VIEW TO [dbatools_dev]  AS [dbo];
GRANT DELETE ON OBJECT::[dbo].[authors] TO [dbatools_dev]  AS [dbo];
GRANT INSERT ON OBJECT::[dbo].[authors] TO [dbatools_dev]  AS [dbo];
"
# Run the exported script
Invoke-DbaQuery -SqlInstance $dbatools2 -SqlCredential $cred -Database $refreshDatabase -Query $grants -Verbose

Remove-DbaDbUser -SqlInstance $dbatools2 -SqlCredential $cred -Database $refreshDatabase -User $loginUser_qa